# Feature Construction

This script constructs the following features:
- Is First-time contributor field
- Second-order degree centrality
- Link intensity

These fields are added to the dataset, removing the fields they are composed of (in case they are aggregated features).
The final dataset is outputted to a `.csv` file.

## Dataset loading

In [77]:
import pandas as pd
import python_proj.utils.exp_utils as exp_utils

input_data_file_name = "non_ftc_data"

data_path_format = exp_utils.BASE_PATH + "/final_data/{data_file_name}.csv"
data_path = data_path_format.format(data_file_name=input_data_file_name)
print(f"{data_path=}")


df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)

print(f"{len(df)=}.")
print(f"{len(df.columns)=}\n")

df.describe()

data_path='/workspaces/msc_thesis/data//final_data/non_ftc_data.csv'
len(df)=1815972.
len(df.columns)=151



,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRIntegratorToSubmitterV2-In),IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToSubmitterV2-In),IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToCommenterV2-In),IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToCommenterV2-In),IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToSubmitterV2-In),...,EcosystemSharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator,EcosystemSharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter,EcosystemExperienceSubmitterIssueSubmissionCount,EcosystemExperienceSubmitterIssueCommentCount,DependencyEcosystemExperienceSubmitterIssueSubmissionCount,DependencyEcosystemExperienceSubmitterIssueCommentCount,NonDependencyEcosystemExperienceSubmitterIssueSubmissionCount,NonDependencyEcosystemExperienceSubmitterIssueCommentCount,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,...,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.474751e+08,5.198276e+06,3.807207e+03,3.190111e+04,4.947501e+00,3.926438e+01,1.076084e+01,1.262702e+01,8.083162e+00,8.484018e+00,...,1.720615e+00,1.699145e+00,2.615631e+00,1.785944e+01,1.743463e-01,8.645552e-01,2.274088e+00,1.570265e+01,1.854588e-01,1.355166e+00
std,1.004294e+08,8.100370e+06,8.071615e+03,1.399335e+05,7.283362e+01,2.497234e+03,3.186353e+02,4.506868e+02,3.652262e+02,1.962613e+02,...,9.780338e+00,9.772010e+00,7.985540e+00,6.700519e+01,1.580686e+00,1.186131e+01,7.064174e+00,6.131080e+01,2.446396e+00,1.667164e+01
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.807322e+07,4.631570e+05,1.000000e+02,5.290000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.371940e+08,1.620265e+06,5.480000e+02,9.799167e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.278026e+08,6.384100e+06,2.817000e+03,7.328350e+03,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,2.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
max,3.617756e+08,5.976123e+07,8.297600e+04,4.489490e+06,1.000000e+04,3.461280e+05,8.789200e+04,1.227920e+05,1.095040e+05,4.764000e+04,...,7.556000e+03,7.556000e+03,2.332000e+03,7.630000e+03,3.730000e+02,9.270000e+02,2.332000e+03,7.630000e+03,7.410000e+02,2.764000e+03


In [78]:
print(list(df.columns))

['ID', 'Project Name', 'Submitter ID', 'PR Number', 'Closed At', 'PullRequestIsMerged', 'SubmitterIsFirstTimeContributor', 'ControlIntegratedBySameUser', 'ControlPullRequestLifeTimeInMinutes', 'ControlPullRequestHasComments', 'ControlNumberOfCommitsInPullRequest', 'ControlPullRequestHasCommentByExternalUser', 'ControlHasHashTagInDescription', 'IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRIntegratorToSubmitterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToSubmitterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToCommenterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToCommenterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToSubmitterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRCommenterToSubmitterV2.PRIntegratorToSubmitterV2-In)', 'IntraProjectSecondOrderDegreeCentrality(PRCommenterToSubmit

## Construct first-time contributor

The FTC field calculated by `sliding_window_3` is by definition wrong (because it's multithreaded and FTC is calculated using a global operator whereas each thread only sees a subset of the data). This error is corrected here.

It can be done in two fashions: 1, with a pre-existing dataset (e.g., one generated by `sliding_window_2`), or 2, by inferring it from the data. It picks a method automatically based on whether a pre-existing file exists. Because inferring it is a little slow, an FTC data file is created, which can be re-used in any future runs.

In [79]:
def replace(
    df1: pd.DataFrame, df2: pd.DataFrame, match_column: str, overwritten_column: str
) -> pd.DataFrame:
    replace_data = df2[[match_column, overwritten_column]]

    old_counts = df1[overwritten_column].value_counts()

    df1 = df1.merge(
        replace_data, how="left", on=match_column, suffixes=("_incorrect", "")
    )
    old_key = f"{overwritten_column}_incorrect"
    df1 = df1.drop(old_key, axis=1)

    new_counts = df1[overwritten_column].value_counts()

    diff = new_counts - old_counts
    print(f"Impact overwriting '{overwritten_column}':\n{diff}")

    return df1


pre_existing_ftc_data_path = f"{exp_utils.BASE_PATH}/final_data/ftc_data.csv"
ftc_key = "SubmitterIsFirstTimeContributor"


def replace_with_pre_existing_data():
    global df
    print(f"{pre_existing_ftc_data_path=}\n")

    other_df: pd.DataFrame = pd.read_csv(pre_existing_ftc_data_path)
    df = replace(df, other_df, "ID", ftc_key)

    del other_df

In [80]:
def replace_without_pre_existing_data():
    """
    Simply tracks any contributor who is not an FTC,
    and if someone is, it checks with the tracked list.
    If they are in the list, this means the variable is,
    wrong and it's set to False.
    """

    global df

    non_ftcs = set()

    total_updated = 0
    for index, row in df.iterrows():
        # It can only illegally go back to FTC,
        # so other cases don't need to be checked.
        is_ftc = row[ftc_key]
        if not is_ftc:
            continue

        project_name = row["Project Name"]
        submitter_id = row["Submitter ID"]
        key = hash(f"{project_name}:{submitter_id}")

        if key in non_ftcs:
            total_updated += 1
            df.at[index, ftc_key] = False
        else:
            non_ftcs.add(key)

    print(f"{total_updated=}")

    ftc_df: pd.DataFrame = df[[*df.columns[:6], ftc_key]]
    ftc_df.to_csv(pre_existing_ftc_data_path, index=False)

In [81]:
from os import path

if path.exists(pre_existing_ftc_data_path):
    print("Using FTC data file.")
    replace_with_pre_existing_data()
else:
    print("Inferring FTC data.")
    replace_without_pre_existing_data()

Using FTC data file.
pre_existing_ftc_data_path='/workspaces/msc_thesis/data//final_data/ftc_data.csv'



Impact overwriting 'SubmitterIsFirstTimeContributor':
SubmitterIsFirstTimeContributor
False    215739
True    -215739
Name: count, dtype: int64


## Construct second-order degree centrality

Calculates 2nd-order degree centrality by aggregating the 2nd-order in-/out-degree values weighing them according to their prevalence, using normalized reciprocal of the edge count; i.e., such that more common edges are weighted less.

_Note, the fields are named 1st-order degree centrality. This is a mistake, but I don't want to rewrite all the data retrieval code._

### Calculate edge weights

In [82]:
import json


# NOTE: There's nothing automated about these numbers;
# I copy-pasted them from the terminal after running ``sliding_window_3.py`` on the dataset.
# The order of these scores matters (the link intensity features assume order).
edge_counts = {
    "PRIntegratorToSubmitterV2": 1043559,
    "PRCommenterToSubmitterV2": 2808889,
    "PRCommenterToCommenterV2": 24595282,
    "IssueCommenterToCommenterV2": 73500466,
    "IssueCommenterToSubmitterV2": 5806894,
}
total_edges = sum(edge_counts.values())


def calculate_weight(value):
    """Calculates normalized complement of the weight."""
    percentage_complement = 1 - (value / (total_edges))
    normalized_perc_compl = percentage_complement / (len(edge_counts) - 1)
    return normalized_perc_compl


edge_weights = {key: calculate_weight(value) for key, value in edge_counts.items()}
total_weight = sum(edge_weights.values())
edge_weights = {key: value / total_weight for key, value in edge_weights.items()}

print("Edge weights:\n", json.dumps(edge_weights, indent=4))

Edge weights:
 {
    "PRIntegratorToSubmitterV2": 0.24757886379195637,
    "PRCommenterToSubmitterV2": 0.24348316399717174,
    "PRCommenterToCommenterV2": 0.19293707610471117,
    "IssueCommenterToCommenterV2": 0.07947333160781547,
    "IssueCommenterToSubmitterV2": 0.23652756449834528
}


### Calculate centrality

In [83]:
from itertools import product

scopes = ["Ecosystem", "IntraProject"]
edge_types = edge_weights.keys()
directions = ["In", "Out"]
combinations = product(scopes, edge_types, edge_types, directions)

BASE_EDGE_FIELD = "{scope}SecondOrderDegreeCentrality({connecting_edge}.{experience_edge}-{direction})"
BASE_TARGET_FIELD = "Weighted{scope}SecondOrder{direction}DegreeCentrality"

centrality_features = []

# Iterates through all edge variables.
for scope, connecting_edge, experience_edge, direction in combinations:
    edge_field = BASE_EDGE_FIELD.format(
        scope=scope,
        connecting_edge=connecting_edge,
        experience_edge=experience_edge,
        direction=direction,
    )

    # Applies the weights to edge.
    connecting_weight = edge_weights[connecting_edge]
    experience_weight = edge_weights[experience_edge]
    total_weight = connecting_weight * experience_weight

    df[edge_field] = df[edge_field].multiply(total_weight)

    # Add contribution to centrality.
    target_field = BASE_TARGET_FIELD.format(scope=scope, direction=direction)
    # Sets initial value if it doesn't exist yet.
    if target_field not in df.columns:
        df[target_field] = 0
        centrality_features.append(target_field)

    df[target_field] = df[target_field].add(df[edge_field])

    # Drops the edge column, as it's only necessary once.
    df = df.drop(edge_field, axis=1)

print(f"{len(df.columns)=}")
df[centrality_features].describe()

len(df.columns)=55


,WeightedEcosystemSecondOrderInDegreeCentrality,WeightedEcosystemSecondOrderOutDegreeCentrality,WeightedIntraProjectSecondOrderInDegreeCentrality,WeightedIntraProjectSecondOrderOutDegreeCentrality
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.258389e+01,1.038033e+01,3.753797e+01,4.061421e+01
std,5.875369e+02,5.176232e+02,1.609692e+03,2.403549e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.567563e-01,4.697693e-02
75%,1.027667e+00,5.467396e-01,2.462693e+00,1.611341e+00
max,2.811220e+05,2.723745e+05,1.035309e+06,1.488838e+06


## Construct Link intensity

Calculates link intensity by aggregating the individual fields, weighting them using the normalized reciprocal of the edge count (the same weights as used for 2nd-order degree centrality).

In [84]:
directions = ["IntegratorToSubmitter", "SubmitterToIntegrator"]

direction_fields = {
    directions[0]: [
        "SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter",
        "SharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter",
        "SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter",
        "SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter",
        "SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter",
    ],
    directions[1]: [
        "SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator",
        "SharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator",
        "SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter",
        "SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter",
        "SharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator",
    ],
}

from wmeijer_utils.collections.list_access import flatten


flt = list(flatten(direction_fields.values()))
flt = [f'{scope}{field}' for scope, field in product(scopes, flt) if f'{scope}{field}' in df.columns]
df[flt].describe()

,EcosystemSharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter,EcosystemSharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter,EcosystemSharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter,EcosystemSharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator,EcosystemSharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator,IntraProjectSharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter,IntraProjectSharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter,IntraProjectSharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator,IntraProjectSharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,6.105694e+00,3.927382e+00,1.699145e+00,4.113105e+00,1.720615e+00,1.091525e+01,5.984465e+00,2.256623e+00,1.278633e+01,7.292949e+00,2.418588e+00
std,3.896921e+01,2.570824e+01,9.772010e+00,2.579123e+01,9.780338e+00,5.254254e+01,2.217244e+01,1.051257e+01,5.283821e+01,2.277851e+01,1.060089e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,1.000000e+01,5.000000e+00,1.000000e+00
max,7.860000e+02,1.021000e+03,7.556000e+03,1.021000e+03,7.556000e+03,1.703000e+03,1.034000e+03,1.824000e+03,1.703000e+03,1.034000e+03,1.824000e+03


In [85]:
BASE_TARGET_FIELD = "{scope}{direction}LinkIntensity"

combinations = product(scopes, directions)
link_intensity_fields = []

edge_weights_list = list(edge_weights.values())
for scope, direction in combinations:
    target_field = BASE_TARGET_FIELD.format(scope=scope, direction=direction)
    fields = direction_fields[direction]

    # Adds initial value for the field.
    df[target_field] = 0
    link_intensity_fields.append(target_field)

    for index, field in enumerate(fields):

        if index in {0, 2, 3}:
            # TODO: REMOVE THIS; THIS SHOULDN'T BE PRESENT IN THE FINAL VERSION.
            continue

        field = f"{scope}{field}"

        # Applies weight
        field_weight = edge_weights_list[index]
        df[field] = df[field].multiply(field_weight)

        # Adds field contribution to link intensity.
        df[target_field] = df[target_field].add(df[field])

        # Drops field as its only used once.
        df = df.drop(field, axis=1)

print(f"{len(df.columns)=}")
df[link_intensity_fields].describe()

len(df.columns)=51


,EcosystemIntegratorToSubmitterLinkIntensity,EcosystemSubmitterToIntegratorLinkIntensity,IntraProjectIntegratorToSubmitterLinkIntensity,IntraProjectSubmitterToIntegratorLinkIntensity
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.358146e+00,1.408445e+00,1.990870e+00,2.347773e+00
std,7.069803e+00,7.091583e+00,6.743430e+00,6.874004e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,2.365276e-01
75%,2.365276e-01,2.434832e-01,9.600215e-01,1.669604e+00
max,1.787202e+03,1.787202e+03,4.319132e+02,4.319132e+02


## Save dataset to file

In [86]:
output_data_file_name = "dataset_with_constructed"
output_path = data_path_format.format(data_file_name=output_data_file_name)
print(f"Outputting to '{output_path}'")

df.to_csv(output_path, index=False)

print(f'{len(df)}')
print(f'{len(df.columns)}')

df.describe()

Outputting to '/workspaces/msc_thesis/data//final_data/dataset_with_constructed.csv'
1815972
51


,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,ControlIntraProjectPullRequestExperienceOfIntegrator,IntraProjectSubmitterPullRequestSubmissionCount,IntraProjectSubmitterPullRequestSuccessRate,IntraProjectSubmitterPullRequestCommentCount,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator,...,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount,WeightedEcosystemSecondOrderInDegreeCentrality,WeightedEcosystemSecondOrderOutDegreeCentrality,WeightedIntraProjectSecondOrderInDegreeCentrality,WeightedIntraProjectSecondOrderOutDegreeCentrality,EcosystemIntegratorToSubmitterLinkIntensity,EcosystemSubmitterToIntegratorLinkIntensity,IntraProjectIntegratorToSubmitterLinkIntensity,IntraProjectSubmitterToIntegratorLinkIntensity
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,...,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.474751e+08,5.198276e+06,3.807207e+03,3.190111e+04,4.947501e+00,1.932087e+02,1.679350e+01,5.352468e-01,2.170048e+01,1.278633e+01,...,1.854588e-01,1.355166e+00,1.258389e+01,1.038033e+01,3.753797e+01,4.061421e+01,1.358146e+00,1.408445e+00,1.990870e+00,2.347773e+00
std,1.004294e+08,8.100370e+06,8.071615e+03,1.399335e+05,7.283362e+01,4.827151e+02,5.548605e+01,4.623260e-01,6.563799e+01,5.283821e+01,...,2.446396e+00,1.667164e+01,5.875369e+02,5.176232e+02,1.609692e+03,2.403549e+03,7.069803e+00,7.091583e+00,6.743430e+00,6.874004e+00
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.807322e+07,4.631570e+05,1.000000e+02,5.290000e+01,1.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.371940e+08,1.620265e+06,5.480000e+02,9.799167e+02,1.000000e+00,3.000000e+01,2.000000e+00,8.000000e-01,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.567563e-01,4.697693e-02,0.000000e+00,0.000000e+00,0.000000e+00,2.365276e-01
75%,2.278026e+08,6.384100e+06,2.817000e+03,7.328350e+03,3.000000e+00,1.410000e+02,1.700000e+01,1.000000e+00,1.500000e+01,1.000000e+01,...,0.000000e+00,0.000000e+00,1.027667e+00,5.467396e-01,2.462693e+00,1.611341e+00,2.365276e-01,2.434832e-01,9.600215e-01,1.669604e+00
max,3.617756e+08,5.976123e+07,8.297600e+04,4.489490e+06,1.000000e+04,4.281000e+03,1.703000e+03,1.000000e+00,2.038000e+03,1.703000e+03,...,7.410000e+02,2.764000e+03,2.811220e+05,2.723745e+05,1.035309e+06,1.488838e+06,1.787202e+03,1.787202e+03,4.319132e+02,4.319132e+02


In [87]:
import datetime

print(datetime.datetime.now())

2024-02-16 15:23:50.177984
